In [25]:
import pandas as pd
import numpy as np
import mysql.connector
import random
import csv

In [31]:
class Episode:
    def __init__(self, id, year):
        self.id = id
        self.year = year
        self.has_recipe = {
            "recipe_name": [], 
            "recipe_cuisine_name": [],
            "chef_id": []
        }
        self.judges = []
        self.grading = {
            "judge_chef_id": [],
            "participant_chef_id": [],
            "grade": []
        }

    def get_has_recipe_df(self):
        hr = {
            "episode_id": self.__get_epid(len(self.has_recipe["recipe_name"])),
            "episode_year": self.__get_epyear(len(self.has_recipe["recipe_name"])),
            "recipe_name": self.has_recipe["recipe_name"],
            "recipe_cuisine_name": self.has_recipe["recipe_cuisine_name"],
            "chef_id": self.has_recipe["chef_id"]
        }
        return pd.DataFrame(hr)

    def get_grading_df(self):
        gr = {
            "episode_id": self.__get_epid(len(self.grading["judge_chef_id"])),
            "episode_year": self.__get_epyear(len(self.grading["judge_chef_id"])),
            "judge_chef_id": self.grading["judge_chef_id"],
            "participant_chef_id": self.grading["participant_chef_id"],
            "grade": self.grading["grade"]
        }
        return pd.DataFrame(gr)

    def get_judges_df(self):
        ju = {
            "episode_id": self.__get_epid(len(self.judges)),
            "episode_year": self.__get_epyear(len(self.judges)),
            "chef_id": self.judges
        }
        return pd.DataFrame(ju)

    def __get_epid(self, length):
        epid = []
        for i in range(0, length): 
            epid.append(self.id)
        return epid
    
    def __get_epyear(self, length):
        epyear = []
        for i in range(0, length):
            epyear.append(self.year)
        return epyear

class Competition:
    def __init__(self, year, cursor):
        self.year = year
        self.cursor = cursor
        self.episodes = []

    def make_episodes(self):
        all_cuisines = self.__get_cuisines()
        recipe_has_chef = self.__get_recipe_has_chef()
        sel_cuisines = random.sample(all_cuisines, 10)

        for episode_id in range(0, 10):
            sel_chefs = []
    
            #rows = []
            ep = Episode(episode_id+1, self.year)
            
            for cui in sel_cuisines:
                available_tuples = []
                for x in recipe_has_chef:
                    if x[1] == cui and not(x[2] in sel_chefs):
                        available_tuples.append((x[0], x[2]))
                sel_tuple = random.choice(available_tuples)
                sel_chefs.append(sel_tuple[1])
                #rows.append((sel_tuple[0], cui, sel_tuple[1]))
                ep.has_recipe["recipe_name"].append(sel_tuple[0])
                ep.has_recipe["recipe_cuisine_name"].append(cui)
                ep.has_recipe["chef_id"].append(sel_tuple[1])
    
            #print(sel_cuisines)
            #print(rows)
            
            all_chef_ids = self.__get_chef_id()
            # remove chefs that are participants
            for sel_id in sel_chefs:
                all_chef_ids.remove(sel_id)
            ep.judges = random.sample(all_chef_ids, 3)
    
            for judge_id in ep.judges:
                for participant_id in ep.has_recipe["chef_id"]:
                    ep.grading["judge_chef_id"].append(judge_id)
                    ep.grading["participant_chef_id"].append(participant_id)
                    ep.grading["grade"].append(random.randint(1, 5))
            self.episodes.append(ep)
    
    def __get_cuisines(self):
        self.cursor.execute("select name from cuisine")
        cuisines = []
        for result in self.cursor:
            cuisines.append(result[0])
        return cuisines

    def __get_chef_id(self):
        self.cursor.execute("select id from chef")
        result = []
        for x in self.cursor:
            result.append(x[0])
        return result
    
    def __get_recipe_has_chef(self):
        self.cursor.execute("select * from recipe_has_chef")
        result = []
        for x in self.cursor: # x = (recipe_name, recipe_cuisine_name, chef_id)
            result.append(x)
            #print(x)
        return result

In [34]:
# Establish connection
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "mydb"
)
mycursor = mydb.cursor()

years = [2022, 2023, 2024]
ep_dict = {
    "id": [],
    "year": [],
    "image_id": []
}
ehc_frames = []
ju_frames = []
gr_frames = []

for year in years:
    comp = Competition(year, mycursor)
    comp.make_episodes()
    
    eps = comp.episodes
    
    for ep in eps:
        ep_dict["id"].append(ep.id)
        ep_dict["year"].append(ep.year)
        ep_dict["image_id"].append(0)
        ehc_frames.append(ep.get_has_recipe_df())
        ju_frames.append(ep.get_judges_df())
        gr_frames.append(ep.get_grading_df())
        
pd.DataFrame(ep_dict).to_csv("episode.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
pd.DataFrame(pd.concat(ehc_frames)).to_csv("episode_has_recipe.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
pd.DataFrame(pd.concat(ju_frames)).to_csv("episode_judges.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')
pd.DataFrame(pd.concat(gr_frames)).to_csv("grading.csv", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"')

mycursor.close()
mydb.close()